In [1]:
!pip install networkx

# Graph Walk
引数:<br>
<b>N</b><br>
タイプ:int,デフォルト:100:生成するタグセットの数<br>
<b>L</b><br>
タイプ:int,デフォルト:2:1つのタグセットにつき生成する最大のタグ数<br>
<b>node_selection</b><br>
タイプ:int,デフォルト:1:0の場合、タグセットの最初のノードを次数に従う確率で選択する。1の場合、優先的選択に従う(初期状態は次数)、2の場合、ランダムに選択する。<br>
<b>edge_selection</b><br>
タイプ:int,デフォルト:1:0の場合、次のノードを隣り合うノードからランダムに選択する。1の場合、優先的選択に従う(初期状態は次数)、2の場合、初期状態で固定する。<br>
<b>initial_edge</b><br>
タイプ:NoneType or nested list of int,デフォルト:None:初期の隣接行列。もしNoneの場合、初期の隣接行列はinitial_Graphに従う。<br>
<b>initial_Graph</b><br>
タイプ:nx.classes.graph.Graph,デフォルト:nx.relaxed_caveman_graph(100,5,0.1):initital_edgeがNoneの場合、こちらに従って、隣接行列を生成する。<br>
<b>おまけ</b>
nx.relaxed_caveman_graph(100,5,0.1)はクリーク数とくりー


In [86]:

class Graph_Walk:
    import random
    import networkx as nx
    def VE(self,st1,X,st2):
        raise ValueError(st1+" should be "+str(st2)+"! "+st1+" = "+str(X))
    def TC(self,s,X,types):#type check
        if type(X) not in types:
            self.VE(s,X,types)
    def TC_list(self,s,ls,types):#type check nest
        for l in ls:
            if type(l) not in types:
                self.VE("Contetnts of "+s,ls,types)
    def TC_nested_list(self,s,ls,types):#type check nest
        for l in ls:
            self.TC_list(s,ls,types)
    def equ_N(self,st,ls):#check equal N
        if self.N==len(ls):
            self.VE("Length of "+st,len(ls),"N")
    def equ_N_nest(self,st,ls):#check equal N to all element ls.
        for i,l in enumerate(ls):
            self.equ_N("Length of "+st+"["+str(i)+"]",l)
    

    def setN(self,N):
        self.TC("N",N,[int])
        self.N=N
    def setL(self,L):
        self.TC("L",L,[int,list])
        if type(L) == list:
            self.TC_list("L",L,[int])
            self.equ_N("L",L)
            self.equ_N_nest("L",L)
            self.TC_nested_list("L",L,[int])
            self.L=L
        else:
            self.L=[L]*self.N
    def setNode_selection(self,NS):
        #0=Edge of randomly selected edge
        #1=PA
        #2=initilal PA
        self.TC("node_selection",NS,[int])
        if NS>=3:
            self.VE("node_selection",NS,"2 or less")
        self.NS=NS
    def setEdge_selection(self,ES):
        #0=RW
        #1=PA
        #2=initilal PA
        self.TC("edge_selection",ES,[int])
        if ES>=3:
            self.VE("edge_selection",ES,"2 or less")
        self.ES=ES
        
    def setInitilal_edge(self,IE):
        #nested list of int
        self.TC("initilal_edge",IE,[list,type(None)])
        if type(IE)==list:
            self.TC_list("initilal_edge",IE,[list])
            self.TC_nested_list("initilal_edge",IE,[int])
        else:
            self.edges=IE
            
    def setInitial_Graph(self,G):
        self.TC("initial_graph",G,[nx.classes.graph.Graph])
        self.graph=G
        self.nodes=G.nodes
        self.edges=nx.adjacency_matrix(G).toarray()
        
    def setNodeVase(self):
        edges=self.edges
        self.NodeVase=[i for i in range(self.N) for j in range(self.N) for k in range(edges[i][j])]
    def setEdgeVase(self):
        edges=self.edges
        self.EdgeVase=[(i,j) for i in range(self.N) for j in range(i,self.N) if edges[i][j]!=0]
        
        
    def __init__(self,N=100,L=2,node_selection=1,edge_selection=1,initial_edge=None,initial_Graph=nx.relaxed_caveman_graph(100,5,0.1)):
        self.setN(N)
        self.setL(L)
        self.setNode_selection(node_selection)#0=Edge,#1=PA,#2=initialPA
        self.setEdge_selection(edge_selection)#0=RW,#1=PA,#2=initialPA
        self.Graph_Flag=0
        
        if initial_edge==None:
            self.setInitial_Graph(initial_Graph)
        else:
            self.setInitilal_edge(initial_edge)
        self.setNodeVase()
        self.setEdgeVase()
        self.generate()
    def PA(self,ls):#
        return self.random.choice(ls)
    def ChoiceNode(self):
        edges=self.edges
        
        if self.NS==0:#Edge
            vase=self.EdgeVase
            edge=self.PA(vase)
            node=self.PA(edge)#PAではないが、実質同じ処理なため、利用。
        elif self.NS==1:#PA
            vase=self.NodeVase
            node=self.PA(vase)
            self.NodeVase.append(node)
        else:
            node=self.random.randint(0,len(edges))
            
        return node
    def RW(self,source):
        edges=self.edges[source]#nodeから伸びているedgeのリストを取得。0なら繋がっていない。
        index_connected_edge=[i for i in range(len(edges)) if edges[i]!=0]
        if self.ES==0:#RW
            node=self.PA(index_connected_edge)
        else:
            connected_edge_vase=[i for i in index_connected_edge for _ in range(edges[i])]
            node=self.PA(connected_edge_vase)
        return node
    def get_tagset(self,i):
        tagset=[self.ChoiceNode()]
        for _ in range(self.L[i]-1):
            tagset.append(self.RW(tagset[-1]))
            if self.NS==1:
                self.NodeVase.append(tagset[-1])
        if self.ES==1:
            for t1,t2 in zip(tagset,tagset[1:]):
                self.edges[t1][t2]+=1
                self.edges[t2][t1]+=1
                    
        return list(set(tagset))
        
        
    def generate(self):
        tagsets=[]
        for i in range(self.N):
            tagsets.append(self.get_tagset(i))
        self.result=tagsets
    def getResult(self):
        return self.result
    def getNodeVase(self):
        return self.NodeVase
    def getEdge(self):
        return self.edges
        
        
        
    
    


In [87]:
GW=Graph_Walk(L=5)
GW.getResult()

[[1, 2, 3, 4],
 [116, 36, 117, 119],
 [45, 46, 47, 48, 49],
 [330, 331, 332, 60, 62],
 [0, 1, 347, 3],
 [0, 1, 3, 4],
 [0, 1, 3, 4],
 [10, 11, 12, 14],
 [330, 331, 332, 334, 62],
 [66, 67, 68, 358],
 [89, 85, 86, 87],
 [80, 81, 82, 84],
 [65, 66, 67, 69],
 [50, 52, 54],
 [50, 52, 53, 54],
 [66, 67, 356, 68, 358],
 [46, 405, 406, 407, 408],
 [24, 20, 22],
 [25, 27, 28, 29],
 [50, 52, 53],
 [370, 372, 374, 26, 27],
 [50, 51, 52, 54],
 [96, 97, 99, 95],
 [0, 1, 3],
 [89, 86, 87],
 [224, 33, 220, 30],
 [15, 16, 17, 18, 342],
 [185, 57, 186, 188, 189],
 [8, 5, 6, 7],
 [49, 45, 47],
 [89, 85, 86, 87],
 [75, 76, 77, 78],
 [100, 60, 102, 103],
 [185, 186, 187, 189, 62],
 [49, 47],
 [75, 76, 77, 78],
 [66, 356, 358],
 [89, 86, 87],
 [66, 67, 356, 68, 358],
 [55, 185, 59, 62, 57],
 [75, 77, 78],
 [5, 6],
 [80, 82, 84],
 [89, 86, 87],
 [0, 4, 50, 52, 53],
 [0, 50, 52, 53],
 [66, 67, 68],
 [66, 67, 358],
 [32, 34, 220, 221, 30],
 [48, 46, 270],
 [187, 188, 189],
 [232, 233, 234, 56, 58],
 [65, 67,

In [66]:
import random
random.choice([0,1])

1

In [54]:
nx.adjacency_matrix(G).toarray()

array([[0, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 1, 0]])